In [ ]:
import os
import json
import csv
import pandas as pd
import numpy as np
import math
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
results_csv_file = r"/Users/admin/Documents/research/active learning/results/results_8_1_2024_lab.csv"

In [ ]:
exp_df = pd.read_csv(results_csv_file)  
ignore_string_in_name_lst = []
include_string_in_name_lst = ["sup"]

In [ ]:
included_exp_names = exp_df['exp'].tolist()

In [ ]:
def get_base_exp_name(exp_name):
    exp_field = "exp"
    exp_field_len_w_index = len(exp_field)+1
    exp_field_index = exp_name.find(exp_field)
    exp_name = exp_name[:exp_field_index]+exp_name[exp_field_index+exp_field_len_w_index:]
    return exp_name

In [ ]:
exp_groups = dict()

for exp_name in included_exp_names:
    base_exp_name = get_base_exp_name(exp_name)
    if base_exp_name in exp_groups:
        exp_groups[base_exp_name] += [exp_name]
    else:
        exp_groups[base_exp_name] = [exp_name]

In [ ]:
exp_length = 5
num_rounds = 10
round_strings =  ["round" + str(round_num) + "_test" for round_num in range(num_rounds)]
base_exp_names = list(exp_groups.keys())

In [ ]:
results_dict = {}

for base_exp_name, exp_group in exp_groups.items():
    continue_flag = False
    for ignore_string_in_name in ignore_string_in_name_lst:
        if ignore_string_in_name in base_exp_name:
            continue_flag = True
            break
    for include_string_in_name in include_string_in_name_lst:
        if include_string_in_name not in base_exp_name:
            print("rejected, ", base_exp_name)
            continue_flag = True
            break
    if continue_flag:
        continue
    exp_dict = {}
    for i, run in enumerate(exp_group):
        sub_df = exp_df.loc[exp_df['exp'] == run]
        for round_string in round_strings:
            val = sub_df[round_string].iloc[0]
            if not math.isnan(val):
                if round_string not in exp_dict:
                    exp_dict[round_string] = [float(val)]
                else:
                    exp_dict[round_string] += [float(val)]
    results_dict[base_exp_name] = exp_dict
    print("accepted ", base_exp_name)

In [ ]:
index = pd.Index(list(results_dict.keys()), name="exp_name")
round_counts_strings = [[round_string, f"round{i}_count"] for i, round_string in enumerate(round_strings)]
mean_strings = round_strings[0:5]
mean_strings = round_strings[1:5]
#mean_strings = round_strings[0:4]
mean_string_col = "mean_test"
df = pd.DataFrame(index=index, 
                  columns=[mean_string_col] + sum(round_counts_strings, []))

In [ ]:
for base_exp_name, results in results_dict.items():
    sub_df = df.loc[base_exp_name]
    results = results_dict[base_exp_name]
    mean_result_list = []
    for round_string, count_string in round_counts_strings:
        if round_string in results:
            round_results = results[round_string]
            num_results = len(round_results)
            mean_result = np.mean(round_results)
            std_results = np.std(round_results, ddof=1)
            mean_string = '%.3f' % mean_result
            std_string = '%.3f'% std_results
            sub_df[round_string] = f"{mean_string}+/-{std_string}"
            sub_df[count_string] = num_results
            if round_string in mean_strings:
                mean_result_list.append(mean_result)
        sub_df[mean_string_col] = np.sum(mean_result_list)/len(mean_result_list)

In [ ]:
df
df.sort_values(by=[mean_string_col], ascending=False)